## Name : Jesus Naranjo
## ID : 1371403


## HW04 Code

You will complete the following notebook, as described in the PDF for Homework 04 (included in the download with the starter code).  You will submit:
1. This notebook file (`hw04.ipynb`), `implementation.py`, and two files for both trees images, i.e., `full`, `full.pdf`, `simple`, and `simple.pdf` (PDFs and text files generated using `graphviz` within the code). HINT: `render()`, and it should be clear when to use it, i.e., #3). Compress all files mentioned and submit to the Gradescope link for code.
2. A PDF of this notebook and all of its output, once it is completed, to the Gradescope link for the PDF.


Please report any questions to the [class Piazza page](https://piazza.com/class/lcwv1h9p2a11ai/).

### Import required libraries.

In [88]:
import numpy as np
import pandas as pd

import sklearn.tree
import graphviz

from implementation import information_remainder, counting_heuristic, set_entropy

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Decision Trees

You should start by computing the two heuristic values for the toy data described in the assignment handout. You should then load the two versions of the abalone data, compute the two heuristic values on features (for the simplified data), and then build decision trees for each set of data.

### 1 Compute both heuristics for toy data.

In [89]:
feature_names = np.array(["A", "B"])
feature_len = 2
classes = [0, 1]

x_set = np.array([[1, 1], [1, 1], [0, 1], [0, 0],
        [0, 1], [0, 0], [0, 0], [0, 0]])
y_set = np.array([0, 0, 0, 0, 1, 1, 1, 1])

#### (a) Compute the counting-based heuristic, and order the features by it.

In [106]:
sort_correct = []  
correct = []
sort_names = []
for i in range(feature_len):
    correct.append(counting_heuristic(x_set, y_set, i, classes))
    
sort_indicies = np.argsort(correct)[::-1]
sort_names = feature_names[sort_indicies]
sort_correct = [correct[i] for i in sort_indicies]

# Print the sorted features along with their correct predictions count in the smaller dataset
longest = max(len(name) for name in sort_names)
for name, correct in zip(sort_names, sort_correct):
    print("%*s: %d/%d" % (longest, name, correct, len(x_set)))

B: 6/8
A: 6/8


#### (b) Compute the information-theoretic heuristic, and order the features by it.

In [105]:
sort_gains = [] 
sort_names_by_gains = []
new_list1 = []
new_list2 = []

for i in range(feature_len):
    sort_gains.append(information_remainder(x_set, y_set, i, classes))
    sort_names_by_gains.append(feature_names[i]) 

for i in np.argsort(sort_gains)[::-1]:
    new_list1.append(sort_gains[i])
    new_list2.append(sort_names_by_gains[i])
sort_gains = new_list1
sort_indices = new_list2

longest = max(len(name) for name in sort_names_by_gains)
for name, gain in zip(sort_names_by_gains, sort_gains):
    print("%*s: %.3f" % (longest, name, gain))

A: 0.311
B: 0.189


#### (c) Discussion of results.

*TODO*

### 2 Compute both heuristics for simplified abalone data.

In [98]:
# full-feature abalone data
x_train = np.loadtxt('data_abalone/x_train.csv', skiprows=1, delimiter=',')
x_test = np.loadtxt('data_abalone/x_test.csv', skiprows=1, delimiter=',')
y_train = np.loadtxt('data_abalone/y_train.csv', skiprows=1, delimiter=',')
y_test = np.loadtxt('data_abalone/y_test.csv', skiprows=1, delimiter=',')

# simplified version of the data (Restricted-feature)
simple_x_train = np.loadtxt('data_abalone/small_binary_x_train.csv', skiprows=1, delimiter=',')
simple_x_test = np.loadtxt('data_abalone/small_binary_x_test.csv', skiprows=1, delimiter=',')
simple_y_train = np.loadtxt('data_abalone/3class_y_train.csv', skiprows=1, delimiter=',')
simple_y_test = np.loadtxt('data_abalone/3class_y_test.csv', skiprows=1, delimiter=',')

# TODO:fix the empty lists below
full_feature_names = np.loadtxt('data_abalone/x_train.csv',delimiter=',', max_rows=1, dtype = str)
simple_feature_names = np.loadtxt('data_abalone/small_binary_x_train.csv',delimiter=',', max_rows=1, dtype = str)
classes_abalone = [0,1,2] # unique set of class labels
class_names = ['small','medium','large'] # name of the classes

#### (a) Compute the counting-based heuristic, and order the features by it.

In [104]:
correct_abalone = []
for i in range(simple_x_train.shape[1]):
    correct_abalone.append(counting_heuristic(simple_x_train, simple_y_train, i, classes_abalone))

sort_correct_abalone, sort_names_abalone = zip(*sorted(zip(correct_abalone, simple_feature_names), reverse=True))

# Print the sorted features along with their correct predictions count in the smaller dataset
longest = max(len(name) for name in sort_names_abalone)
for name, correct in zip(sort_names_abalone, sort_correct_abalone):
    print("%*s: %d/%d" % (longest, name, correct, len(simple_x_train)))

height_mm: 2316/3176
  diam_mm: 2266/3176
length_mm: 2230/3176
  is_male: 1864/3176


#### (b) Compute the information-theoretic heuristic, and order the features by it.

In [110]:
sort_gains_abalone = []
sort_names_by_gains_abalone = []

for i in range(len(simple_feature_names)):
    sort_gains_abalone.append(information_remainder(simple_x_train, simple_y_train, i, classes_abalone))
    sort_names_by_gains_abalone.append(simple_feature_names[i]) 
    
sort_indices = np.argsort(sort_gains_abalone)[::-1]
sort_gains_abalone = [sort_gains_abalone[i] for i in sort_indices]
sort_names_by_gains_abalone = [sort_names_by_gains_abalone[i] for i in sort_indices]

longest = max(len(name) for name in sort_names_by_gains_abalone)
for name, gain in zip(sort_names_by_gains_abalone, sort_gains_abalone):
    print("%*s: %.3f" % (longest, name, gain))

height_mm: 0.173
  diam_mm: 0.150
length_mm: 0.135
  is_male: 0.025


### 3) Generate decision trees (criterion='entropy', random_state=42) for full- and simple-feature data

#### (a) Train and eval on entire train and test sets. Print accuracy values and generate tree images.

Render the tree diagram, naming it "full." A text file and PDF should be created and saved (i.e., `full` and `full.pdf`) - include both in submission.

In [112]:
clf = sklearn.tree.DecisionTreeClassifier(criterion='entropy', random_state=42)

clf.fit(x_train, y_train)
train_accuracy = clf.score(x_train, y_train)
test_accuracy = clf.score(x_test, y_test)
print(f"Accuracy (train): {train_accuracy:.3f}")
print(f"Accuracy  (test): {test_accuracy:.3f}")

# tree image
tree = sklearn.tree.export_graphviz(clf, feature_names=full_feature_names, filled = True)
graph1 = graphviz.Source(tree)
graph1.render("full")

Accuracy (train): 1.000
Accuracy  (test): 0.204



(process:31672): GLib-GIO-WARNING **: 21:17:21.260: Unexpectedly, UWP app `Clipchamp.Clipchamp_2.5.15.0_neutral__yxz26nhyzhsrt' (AUMId `Clipchamp.Clipchamp_yxz26nhyzhsrt!App') supports 41 extensions but has no verbs

(process:31672): GLib-GIO-WARNING **: 21:17:21.366: Unexpectedly, UWP app `Microsoft.ScreenSketch_11.2302.20.0_x64__8wekyb3d8bbwe' (AUMId `Microsoft.ScreenSketch_8wekyb3d8bbwe!App') supports 29 extensions but has no verbs


'full.pdf'

#### (b) Restricted-feature (aka simple) data.
Train and eval on simple train and test sets. Same as above, accept this time use the `simple` set. Render the tree diagram, naming it "simple." A text file and PDF should be created and saved (i.e., `simple` and `simple.pdf`) - include both in submission.

In [113]:
simple_clf = sklearn.tree.DecisionTreeClassifier(criterion='entropy', random_state=42)

simple_clf.fit(simple_x_train, simple_y_train)
simple_train_accuracy = simple_clf.score(simple_x_train, simple_y_train)
simple_test_accuracy = simple_clf.score(simple_x_test, simple_y_test)
print(f"Accuracy (train): {simple_train_accuracy:.3f}")
print(f"Accuracy  (test): {simple_test_accuracy:.3f}")

# tree image
simple_tree = sklearn.tree.export_graphviz(simple_clf, feature_names=simple_feature_names, class_names=class_names, filled = True)
graph2 = graphviz.Source(simple_tree)
graph2.render("simple")

Accuracy (train): 0.733
Accuracy  (test): 0.722


'simple.pdf'

#### (c) Discuss the results seen for the two trees

*TODO*